In [1]:
# !git clone 'https://github.com/crispitagorico/Signature-Kernel.git'

In [1]:
# %cd ../src

In [2]:
# !python setup.py build_ext --inplace

In [1]:
import sys
sys.path.append('../src')
sys.path.append('../utils')

import torch
import time
import numpy as np

from sigLoss import SigLoss, SigLoss_naive
from sigMMD import SigMMD, SigMMD_naive

import matplotlib.pyplot as plt

# Sig Loss gradients

In [40]:
A = 1
M = 2
N = 2
D = 2

X = np.random.randn(A,M,D).cumsum(axis=1)
Y = np.random.randn(A,N,D).cumsum(axis=1)

X /= np.max(X)
Y /= np.max(Y)

In [74]:
X_naive = torch.tensor(X, dtype=torch.float64)
Y_naive = torch.tensor(Y, dtype=torch.float64)

X_cpu = X_naive.clone()
Y_cpu = Y_naive.clone()

# X_gpu = X_naive.clone().cuda()
# Y_gpu = Y_naive.clone().cuda()

X_naive.requires_grad = True
X_cpu.requires_grad = True
# X_gpu.requires_grad = True

n = 6
solver = 2

In [60]:
t = time.time()
l_naive = SigLoss_naive(n=n,solver=solver).forward(X_naive,Y_naive)
print('time:', np.round(time.time()-t,3), 's')
print(l_naive)

time: 0.153 s
tensor(29.0358, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [75]:
t = time.time()
l_cpu = SigLoss(n=n,solver=solver).forward(X_cpu,Y_cpu)
print('time:', np.round(time.time()-t,3), 's')
print(l_cpu)

time: 0.003 s
tensor(28.9584, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [76]:
# t = time.time()
# l_gpu = SigLoss(n=n,solver=solver).forward(X_gpu,Y_gpu)
# print('time:', np.round(time.time()-t,3), 's')
# print(l_gpu)

In [63]:
t = time.time()
l_naive.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.15 s


In [77]:
t = time.time()
l_cpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.003 s


In [72]:
# t = time.time()
# l_gpu.backward()
# print('time:', np.round(time.time()-t,3), 's')

In [66]:
X_naive.grad

tensor([[[-6.7105, -5.3226],
         [ 6.7105,  5.3226]]], dtype=torch.float64)

In [78]:
X_cpu.grad

tensor([[[-6.5416, -5.2685],
         [ 6.5416,  5.2685]]], dtype=torch.float64)

In [11]:
torch.allclose(X_naive.grad, X_cpu.grad)

False

In [12]:
torch.allclose(X_naive.grad, X_gpu.grad.cpu())

NameError: name 'X_gpu' is not defined

In [63]:
torch.allclose(X_cpu.grad, X_gpu.grad.cpu())

True

# Sig MMD gradients

In [64]:
A = 3
B = 2
M = 7
N = 7
D = 4

X = np.random.randn(A,M,D).cumsum(axis=1)
Y = np.random.randn(B,N,D).cumsum(axis=1)

X /= np.max(X)
Y /= np.max(Y)

X_naive = torch.tensor(X, dtype=torch.float64)
Y_naive = torch.tensor(Y, dtype=torch.float64)

X_cpu = X_naive.clone()
Y_cpu = Y_naive.clone()

X_gpu = X_naive.clone().cuda()
Y_gpu = Y_naive.clone().cuda()

X_naive.requires_grad = True
X_cpu.requires_grad = True
X_gpu.requires_grad = True

In [65]:
t = time.time()
mmd_naive = SigMMD_naive(n=n,solver=solver).forward(X_naive,Y_naive)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_naive)

time: 0.149 s
tensor(1.7695, dtype=torch.float64, grad_fn=<SubBackward0>)


In [66]:
t = time.time()
mmd_cpu = SigMMD(n=n,solver=solver).forward(X_cpu,Y_cpu)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_cpu)

time: 0.001 s
tensor(1.7695, dtype=torch.float64, grad_fn=<SubBackward0>)


In [67]:
t = time.time()
mmd_gpu = SigMMD(n=n,solver=solver).forward(X_gpu,Y_gpu)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_gpu)

time: 0.005 s
tensor(1.7695, device='cuda:0', dtype=torch.float64, grad_fn=<SubBackward0>)


In [68]:
t = time.time()
mmd_naive.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.143 s


In [69]:
t = time.time()
mmd_cpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.002 s


In [70]:
t = time.time()
mmd_gpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.006 s


In [71]:
torch.allclose(X_naive.grad, X_cpu.grad)

False

In [72]:
torch.allclose(X_naive.grad, X_gpu.grad.cpu())

False

In [73]:
torch.allclose(X_cpu.grad, X_cpu.grad)

True